In [1]:
# importing all necessary packages
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [4]:
X = pd.read_csv('training_set_rel3.csv', sep='\t',encoding='ISO-8859-1')

In [5]:
y = X['domain1_score']
# print(y)
X = X.dropna(axis=1)
X=X[X['essay_set']==1]
X = X.drop(columns=['rater1_domain1', 'rater2_domain1'])

In [6]:
X.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [7]:
import torch
import transformers

In [8]:
from transformers import GPT2LMHeadModel
from transformers import GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained('gpt2')  
word_embeddings = model.transformer.wte.weight  
position_embeddings = model.transformer.wpe.weight
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [9]:
import numpy as np
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from nltk.tokenize import PunktSentenceTokenizer
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

C:\Users\Bhaskar Rao\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to C:\Users\Bhaskar
[nltk_data]     Rao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
def essay_to_sentences(essay_v):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    #tokennizer=PunktSentenceTokenizer(english.pickle)
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        #removing empty sentences
        if len(raw_sentence) > 0:
            sentences.append((raw_sentence))
    #print(sentences)
    return sentences

def makeFeatureVec(words, num_features,n):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((n,num_features),dtype="float32")    
    text_index = tokenizer.encode(words,add_prefix_space=True)
    x = model.transformer.wte.weight[text_index,:]
    x=torch.tensor(x,requires_grad=True)
    x=x.detach().numpy()
    y=len(x[0])
    featureVec = np.add(featureVec,x).flatten()
    return featureVec

def getAvgFeatureVecs(essays, num_features,aa):
    """Main function to generate the word vectors for GPT2 model"""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features*96),dtype="float32")
    for essay in essays:
        arr=makeFeatureVec(essay, num_features,aa[counter]).flatten() 
        ss=(num_features*96)-len(arr)
        arr1=np.pad(arr,(0,ss),'constant', constant_values=(0, 0))
        essayFeatureVecs[counter] = np.add(arr1,np.zeros((num_features*96),dtype="float"))
        counter = counter + 1
    return essayFeatureVecs

In [11]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from keras.layers import Bidirectional

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(Bidirectional(LSTM(300, return_sequences=True),input_shape=[1, 73728])) #96*128
    model.add(Bidirectional(LSTM(300, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True)))
    model.add(Bidirectional(LSTM(64, recurrent_dropout=0.4)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])
    model.summary()

    return model


In [12]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score
import tensorflow as tf
from numpy import array
import tensorflow.compat.v1 as tf1
import torch

cv = KFold(n_splits =5, shuffle = True)
results = []
results2 = []
y_pred_list = []

count = 1
for traincv, testcv in cv.split(X):
    print("\n--------Fold {}--------\n".format(count))
    X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
    #print(y_train)
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    num_features = 768
    clean_train_essays = []
    clean_test_essays=[]
    #essays=train_essays
    sentences=[]
    l1=[]
    l2=[]
    for essay in train_essays:
      sentences += essay_to_sentences(essay)
  
      #print(sentences)
    for essay in test_essays:
      sentences += essay_to_sentences(essay)

    for essay_v in train_essays:
      l1.append(len(sent_tokenize(essay_v)))
      clean_train_essays.append(essay_to_sentences(essay_v))
        
      #print(clean_train_essays)
    a=np.array(l1) ################
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, num_features,a)
    for essay_v in test_essays:
      l2.append(len(sent_tokenize(essay_v)))
      clean_test_essays.append(essay_to_sentences(essay_v))
    b=np.array(l2)
    
    testDataVecs=getAvgFeatureVecs(clean_test_essays, num_features,b)
    trainDataVecs=np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    trainDataVecs=np.reshape(trainDataVecs,(trainDataVecs.shape[0],1,trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    lstm_model = get_model()
    history=lstm_model.fit(trainDataVecs, y_train, batch_size=60, epochs=10)
    y_pred=(lstm_model.predict(testDataVecs))
    y_trained=(lstm_model.predict(trainDataVecs))
    
    
    
    lstm_model.save('model.h5')
    # Save any one of the 5 models.
#     if count == 6:
    if count==6:
        break
    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)
    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    
    print("Kappa Score of Testing: {}".format(result))
    results.append(result)
    count += 1
   
    


print("Average Kappa score after a 5-fold cross validation: ",np.around(np.array(results).mean(),decimals=4))



--------Fold 1--------



C:\Users\Bhaskar Rao\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 1, 600)            177669600 
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1, 600)            2162400   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               340480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 180,172,609
Trainable params: 180,172,609
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
24/24 [==============================] - 99s 2s/step - loss: 20.3038 - accuracy: 0.0000e+00
Epoch